<a href="https://colab.research.google.com/github/RomaricKanyamibwa/Normalisation-de-concepts-medicaux/blob/master/CNN_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt
import logging


Using TensorFlow backend.


In [0]:
!pip3 install --upgrade gensim

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
from gensim.models import Word2Vec
import gensim
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np

In [6]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/Word2VecModels/
!ls -l drive/Coolab/

model_2_LivingBeingsTrainingSet.bin
modelLivingBeingsTrainingSet.bin
modelPourRomaricLivingBeingsTrainingSet.bin
total 818310
-rw-r--r-- 1 root root     30397 Dec 14 19:51 CNN_projet.ipynb
-rw-r--r-- 1 root root  20688992 Dec 13 20:53 Disorders
-rw-r--r-- 1 root root  62187545 Nov 25 22:57 Disorders.csv
-rw-r--r-- 1 root root 377493728 Dec 13 20:53 Disorders.trainables.syn1neg.npy
-rw-r--r-- 1 root root 377493728 Dec 13 20:53 Disorders.wv.vectors.npy
-rw-r--r-- 1 root root      8864 Dec  8 20:50 loadmodel.ipynb
drwxr-xr-x 2 root root      4096 Dec  8 17:30 sg-s0200-w08-m004-ns05-s0.001-a0.025-i05
-rw-r--r-- 1 root root     17800 Dec 11 10:19 visualize.ipynb
-rw-r--r-- 1 root root     21012 Dec 13 22:28 word2vec.ipynb


In [44]:
google_drive="drive/Coolab/"
df_disorders=pd.read_csv("drive/Coolab/Disorders.csv",names=["Code","Disorder","GenDisorder"],sep="\t") 
num_rows=df_disorders.shape[0]

#step 2:Clean data
count_empty=df_disorders.isnull().sum()
count_non_empty=count_empty[count_empty==0]
print(count_non_empty)
#remove empty rows
df_disorders=df_disorders[count_non_empty.keys()]

Code           0
Disorder       0
GenDisorder    0
dtype: int64


In [69]:
Codes=df_disorders['Code']#Y vector (outputs)
Disorders=df_disorders['Disorder']#X vector(test)
GenDisorders=df_disorders['GenDisorder']#X vector (train)
print(len(Disorders))
print(len(Codes))

623509
623509


In [0]:
sample_size=15000#Disorders.size//10
#train
Disorders=Disorders.sample(sample_size,random_state=10)
Codes=Codes[Disorders.index]


In [71]:
#test set
tsample_size=sample_size//5
testDisorders=df_disorders['Disorder']
testDisorders=testDisorders.drop(Disorders.index)
testDisorders=testDisorders.sample(tsample_size,random_state=10)
testCodes=df_disorders['Code']
testCodes=testCodes[testDisorders.index]
len(testCodes)

3000

In [48]:
print(Disorders.index)

Int64Index([222160,  56824, 426769, 154154, 173255, 125527, 183515, 548570,
            552073, 249167,
            ...
            515022,  72110, 344502,  74068, 271092, 161909, 552885, 470888,
            304302, 451694],
           dtype='int64', length=15000)


In [49]:
for i,j in enumerate(zip(Codes,Disorders)):
  print(j)
  if(i>10):break

('C0423346', 'morax-axenfeld; conjunctivitis')
('C4510595', 'malposition; congenital, clavicle')
('C1394389', 'self-inflicted overdose of anterior pituitary hormones')
('C1394389', 'headache syndrome, hypnic')
('C1394389', 'gonococcal septicaemia affecting skin')
('C1394389', 'disease (or disorder); pulmonary, vessels')
('C1394389', 'cryptococcus neoformans; infection')
('C1394389', 'abuse aged')
('C0520729', 'other natural environment type phobia')
('C0332603', 'acute haematogenous osteomyelitis, multiple sites')
('C0333292', 'cellulitis of right thumb')
('C0333292', 'driver of three-wheeled motor vehicle injured in collision with car, pick-up truck or van in nontraffic accident, sequela')


In [50]:
for i,j in enumerate(zip(testCodes,testDisorders)):
  print(j)
  if(i>10):break

('C0553642', 'nonarticular rheumatism')
('C2841666', 'nondisp fx of lateral epicondyl of l humer, 7thg')
('C4024440', 'fragmentation of end part of the middle bone of the middle finger')
('C0475604', 'tracheobronchial hemorrhage origin in the perinatal period')
('C0349318', 'unspecified mental retardation with the statement of no, or minimal, impairment of behavior')
('C2903289', 'oophoritis following (induced) termination of pregnancy')
('C2867100', 'nondisplaced fracture of anterior process of left calcaneus, subsequent encounter for fracture with nonunion')
('C0232066', 'induced apnea')
('C2883613', 'poisoning by electrolytic, caloric and water-balance agents, assault')
('C2103537', 'trauma of left eye')
('C0375046', 'pl cocidioidomycosis nos')
('C2904131', 'walked into furniture')


In [0]:
def removed_elements_generator(input_list):
    """This function creates an id generator of the empty tokenized elements"""
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line_process=remove_stopwords(line)
        line_process=gensim.utils.simple_preprocess(line_process,min_len=3,max_len=30)
        if not line_process:
          print(line_process,"before preprocess-",i,":",line)
          RemovedElements.append(i)
          #line_process=line.split()
          yield i

def input_tokenize(input_list,RemovedElements):
    """This creates a generator of the tokenized elements """
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line_process=remove_stopwords(line)
        line_process=gensim.utils.simple_preprocess(line_process,min_len=3,max_len=30)
        if not line_process:
          print(line_process,"before preprocess-",i,":",line)
          RemovedElements.append(i)
          #line_process=line.split()
        else:    
          yield line_process

In [0]:
logging.info ("Tokenization")
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
#GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("End of Tokenization")

In [53]:
for i,s in enumerate(Disorders_corpus):
  #print(s)
  k=1
  #if i >:break
    
print(RemovedElements)

[] before preprocess- 469 : ad18
[] before preprocess- 767 : hs5
[] before preprocess- 1235 : t(17;22)(q22;q13)
[] before preprocess- 1238 : rp69
[] before preprocess- 2640 : t(17;21)(q11;q22)
[] before preprocess- 3070 : ts
[] before preprocess- 4897 : ml i
[] before preprocess- 4960 : rp15
[] before preprocess- 6175 : cd332 c.1645a>c
[] before preprocess- 7334 : cd332 c.1144t>c
[] before preprocess- 8473 : ad5
[] before preprocess- 8543 : t(2;17)(p23;q35)
[] before preprocess- 8678 : t2d
[] before preprocess- 9224 : k20r
[] before preprocess- 10574 : fire in mine
[] before preprocess- 11674 : dn
[] before preprocess- 13630 : ea4
[] before preprocess- 14399 : rp73
[] before preprocess- 14671 : oi9
[] before preprocess- 14865 : md1
[469, 767, 1235, 1238, 2640, 3070, 4897, 4960, 6175, 7334, 8473, 8543, 8678, 9224, 10574, 11674, 13630, 14399, 14671, 14865]


In [0]:
model_dir=google_drive+"Disorders"
model=Word2Vec.load(model_dir)


In [0]:
def getsentence_matrix(sentence,gensim_model,nrows=30):
  #print("sent:",sentence)
  sentence_matrix=[gensim_model.wv.get_vector(token) for token in sentence]
 
  sentence_matrix=np.matrix(sentence_matrix)
#  print("Before padding:")
#  pprint(sentence_matrix.T[:2,:])
  #print("size before padding:",nrows-sentence_matrix.shape[0])
  if(nrows-sentence_matrix.shape[0]<0 or sentence_matrix.shape[1]<=0):
    print("Sentence:",sentence)
    print("MatrixShape:",sentence_matrix.shape)
    print(sentence_matrix)
  sentence_matrix=np.lib.pad(sentence_matrix,((0,max(0,nrows-sentence_matrix.shape[0])),(0,0)), 'constant', constant_values=(0))
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
#  print("After padding:")
#  pprint(sentence_matrix.T[:2,:])
  #print("shape=",sentence_matrix.shape)
  return sentence_matrix#.T

In [0]:
def get_termes_matrices(Corpus,model):
  
#  pprint(Corpus)
  matrix_list=[]
  IndexOfEmptyElem=[]
  nrows=30
  empty=True
  for i,sentence in enumerate(Corpus):
    if not(not sentence):
      if empty==True:
        matrix_list=np.array([getsentence_matrix(sentence,model,nrows=30)])
        empty=False
        #print(matrix_list[0].shape)
      else:
        #print(i,":",getsentence_matrix(sentence,model).shape)
        matrix_list=np.append(matrix_list,[getsentence_matrix(sentence,model,nrows=30)],axis=0)
      #print("i:",i)#,matrix_list)
      if(i%1000==0):
        print("i:",i)
       # break
    else:
      print("Empty element's index:",i)
      IndexOfEmptyElem.append(i)
      #print(np.zeros((5,5)))
      matrix_list=np.append(matrix_list,[np.zeros((nrows,model.wv.vector_size))],axis=0)
  print("length:",len(matrix_list))
  print("shape=",matrix_list.shape)
  #print(matrix_list)
  return matrix_list,IndexOfEmptyElem


In [0]:
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
testRemovedElements=[]
testDisorders_corpus=(input_tokenize(testDisorders,testRemovedElements))

In [0]:
RemovedElementsIter=removed_elements_generator(Disorders)
testRemovedElementsIter=removed_elements_generator(testDisorders)

In [59]:
print(list(RemovedElementsIter))
print("Test")
print(list(testRemovedElementsIter))

[] before preprocess- 469 : ad18
[] before preprocess- 767 : hs5
[] before preprocess- 1235 : t(17;22)(q22;q13)
[] before preprocess- 1238 : rp69
[] before preprocess- 2640 : t(17;21)(q11;q22)
[] before preprocess- 3070 : ts
[] before preprocess- 4897 : ml i
[] before preprocess- 4960 : rp15
[] before preprocess- 6175 : cd332 c.1645a>c
[] before preprocess- 7334 : cd332 c.1144t>c
[] before preprocess- 8473 : ad5
[] before preprocess- 8543 : t(2;17)(p23;q35)
[] before preprocess- 8678 : t2d
[] before preprocess- 9224 : k20r
[] before preprocess- 10574 : fire in mine
[] before preprocess- 11674 : dn
[] before preprocess- 13630 : ea4
[] before preprocess- 14399 : rp73
[] before preprocess- 14671 : oi9
[] before preprocess- 14865 : md1
[469, 767, 1235, 1238, 2640, 3070, 4897, 4960, 6175, 7334, 8473, 8543, 8678, 9224, 10574, 11674, 13630, 14399, 14671, 14865]
Test
[] before preprocess- 217 : t(11;14)
[] before preprocess- 1556 : ss
[] before preprocess- 1647 : xp-a
[] before preprocess- 227

In [0]:
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
testRemovedElements=[]
testDisorders_corpus=(input_tokenize(testDisorders,testRemovedElements))


In [61]:
MatrixList,empty=get_termes_matrices(Disorders_corpus,model)

i: 0
[] before preprocess- 469 : ad18
[] before preprocess- 767 : hs5
i: 1000
[] before preprocess- 1235 : t(17;22)(q22;q13)
[] before preprocess- 1238 : rp69
i: 2000
[] before preprocess- 2640 : t(17;21)(q11;q22)
i: 3000
[] before preprocess- 3070 : ts
i: 4000
[] before preprocess- 4897 : ml i
[] before preprocess- 4960 : rp15
i: 5000
i: 6000
[] before preprocess- 6175 : cd332 c.1645a>c
i: 7000
[] before preprocess- 7334 : cd332 c.1144t>c
i: 8000
[] before preprocess- 8473 : ad5
[] before preprocess- 8543 : t(2;17)(p23;q35)
[] before preprocess- 8678 : t2d
i: 9000
[] before preprocess- 9224 : k20r
i: 10000
[] before preprocess- 10574 : fire in mine
i: 11000
[] before preprocess- 11674 : dn
i: 12000
i: 13000
[] before preprocess- 13630 : ea4
i: 14000
[] before preprocess- 14399 : rp73
[] before preprocess- 14671 : oi9
[] before preprocess- 14865 : md1
length: 14980
shape= (14980, 30, 300)


In [62]:
print("test sample")
testMatrixList,testempty=get_termes_matrices(testDisorders_corpus,model)

test sample
i: 0
[] before preprocess- 217 : t(11;14)
i: 1000
[] before preprocess- 1556 : ss
[] before preprocess- 1647 : xp-a
i: 2000
[] before preprocess- 2276 : t(3;12)(q27-q28;q13-q15)
[] before preprocess- 2605 : dd
[] before preprocess- 2820 : hs1
length: 2994
shape= (2994, 30, 300)


In [0]:
def remove_element_byIndex(panda_dataset,listOfIndexes):
  Indexes=[]
  for elem in listOfIndexes:
    Indexes.append(panda_dataset[elem:elem+1].index[0])
  
  print(Indexes)
  return panda_dataset.drop(Indexes)

In [72]:


print(Codes.reindex([219126, 267602]))
#print(Disorders[RemovedElements[0]:RemovedElements[0]+1])
print(Codes[RemovedElements[0]:RemovedElements[0]+1])
#print(Disorders[RemovedElements[1]:RemovedElements[1]+1])
#po=pd.Int64Index([], dtype='int64')
#po.append(224267)

print("Before drop:",len(Codes))
Codes2=remove_element_byIndex(Codes,RemovedElements)
print("After drop:",len(Codes2))
print(testempty)
print("after drop")

219126    C3810041
267602    C2675192
Name: Code, dtype: object
219126    C3810041
Name: Code, dtype: object
Before drop: 15000
[219126, 267602, 26735, 284050, 23305, 323019, 124679, 231113, 24110, 26207, 314043, 30167, 67531, 29851, 381658, 315505, 110224, 151841, 125081, 563840]
After drop: 14980
[]
after drop


In [73]:
print("Before drop:",len(testCodes))
testCodes2=remove_element_byIndex(testCodes,testRemovedElements)
print("After drop:",len(testCodes2))

Before drop: 3000
[24260, 121969, 343403, 25037, 43115, 87803]
After drop: 2994


In [0]:
def get_code_classes(Codes,gensim_model):
  unique_Codes=list(set(Codes))
  nb_classes=len(unique_Codes)
  print("number of codes:",nb_classes)
  Code_vectors=[gensim_model.wv.get_vector(token) for token in Codes]
  Code_dict=dict(zip(Codes,Code_vectors))
  #inv_Code_dict=dict(zip(Code_vectors,Codes))
  #print(Code_vectors[0][0:5])
  #print("'C0009021'",Code_dict['C0009021'][0:5])
  return nb_classes,Code_vectors,Code_dict

In [75]:
nb_classes,Code_vectors,Code_dict=get_code_classes(Codes2,model)

number of codes: 14059


In [76]:
Codes[222160]
print("Dict:",Code_dict["C0275585"][0:5])
print("Vect:",Code_vectors[0][0:5])

Dict: [-0.08187973 -0.05139588 -0.13897417 -0.08761195 -0.03717935]
Vect: [-0.08187973 -0.05139588 -0.13897417 -0.08761195 -0.03717935]


In [77]:
testnb_classes,testCode_vectors,testCode_dict=get_code_classes(testCodes2,model)

#testCodes[244939]
#print("Dict:",testCode_dict["C0341679"][0:5])
#print("Vect:",testCode_vectors[0][0:5])

number of codes: 2950


In [132]:
frames = [Codes2,testCodes2]
result = pd.concat(frames)

print(len(((result))))

print( len( result[Codes2.index] ) )

17974
14980


In [0]:
print(result)

In [116]:
batch_size = 128
num_classes = nb_classes
epochs = 10

# input dimensions
size_x, size_y = MatrixList[0].shape
keras_model = Sequential()
print(type(np.array(MatrixList)))

# load the MNIST data set, which already splits into train and test sets for us
x_train, y_train = np.array(MatrixList),np.array(Codes2)
x_test, y_test = np.array(testMatrixList),np.array(testCodes2)
print(type(x_train[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [117]:
print((x_train.shape))
print((x_test.shape))
print((y_train.shape))
print((y_test.shape))

(14980, 30, 300)
(2994, 30, 300)
(14980,)
(2994,)


In [0]:
# reshape the data into a 4D tensor - (sample_number, x_size, y_size, num_channels)
# we only have a single channel
x_train = x_train.reshape(x_train.shape[0], size_x, size_y, 1)
x_test = x_test.reshape(x_test.shape[0], size_x, size_y, 1)
input_shape = (size_x, size_y, 1)

In [119]:
# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (14980, 30, 300, 1)
14980 train samples
2994 test samples


In [0]:
# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
y_train = y_train.reshape(y_train.shape[0], 1)#keras.utils.to_categorical(y_train, nb_classes)
y_test = y_test.reshape(y_test.shape[0], 1)#keras.utils.to_categorical(y_test, testnb_classes)


In [124]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

print((y_train.shape))
print((y_test.shape))

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

encoder.fit(y_test)
encoded_Ytest = encoder.transform(y_test)
y_test = np_utils.to_categorical(encoded_Ytest)

#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

print((y_train.shape))
print((y_test.shape))

(14980, 14059)
(2994, 2950)


ValueError: ignored

In [123]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
plt.plot(range(1, 11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

ValueError: ignored